<a href="https://colab.research.google.com/github/ferrap/LLM_ETH/blob/main/Astra_DB_%2B_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install **DataStax Python** driver

In [ ]:
pip install cassandra-driver

Verify Python driver installation and version

In [ ]:
import cassandra
print (cassandra.__version__)

# Connect to Astra DB using the **Secure Connect Bundle** and setup the cluster connection



In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config= {
  'secure_connect_bundle': '/content/drive/MyDrive/secure-connect-workshops.zip'
}
auth_provider = PlainTextAuthProvider('KcZsXDTwQkbvhMgqIJzSnaTk', 'rz8gHPnIf74ftqeotc734Fg9dHK4uWSo5G5EsJb9nz+kO6aKE9yqZPnJhQ1l2QcqZOw6QIXSDGQQcBtwpBALEN9R.jWDc_CWaI+nc.s0hauRwnT8pg2ePsmPAOlhqD49')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)

#USE the ecommerce keyspace
session = cluster.connect("ecommerce")


Verify connection to Astra DB checking the cluster name, version and number of nodes

In [ ]:
cass_output_1 = session.execute("SELECT cluster_name, release_version FROM system.local")
for cass_row in cass_output_1:
  output_message = "Connected to " + str(cass_row.cluster_name) + " and it is running " + str(cass_row.release_version) + " version."
print(output_message)

cass_output_2 = session.execute("SELECT count(1) AS nodes_count FROM system.peers")
for cass_row in cass_output_2:
  output_message = "This cluster contains " + str(cass_row.nodes_count+1) + " nodes."
print(output_message)

# Create new table *cart_products* in the CQL console on Astra DB



```
CREATE TABLE IF NOT EXISTS cart_products (
    cart_id uuid,
    product_timestamp timestamp,
    product_name text,
    quantity int,
    product_description text,
    PRIMARY KEY (cart_id, product_name)
);
```







# Simulate shopping cart data

Use *BatchStatement* to run multiple statements (**INSERT**) into a single atomic transaction

In [ ]:
import uuid
import random
import datetime
import time
from cassandra.query import BatchStatement


# opening the file in read mode
products_txt = open("/content/drive/MyDrive/fruits.txt", "r")
descriptions_txt = open("/content/drive/MyDrive/descriptions.txt", "r")
# reading the file
data_products = products_txt.read()
data_descriptions = descriptions_txt.read()
# replacing end splitting the text when newline ('\n') is seen.
fruits_list = data_products.split("\n")
descriptions_list = data_descriptions.split("\n")

cart_id = uuid.uuid4()
insertCart = 'INSERT INTO cart_products (cart_id,product_timestamp,product_name,quantity,product_description) VALUES (?,?,?,?,?)'

print('========================================')

try:

  prepared_insertCart = session.prepare(insertCart)
  batch = BatchStatement()

  for name, description in zip(fruits_list,descriptions_list):

    product_timestamp = datetime.datetime.now()
    quantity = random.randrange(1,5)
    product_name = name
    product_description = description

    batch.add(prepared_insertCart, [cart_id,product_timestamp,product_name,quantity,product_description])
    print(str(quantity) + ' ' + product_name + ' added to cart at '+ str(product_timestamp) + ' with Success!')

    time.sleep(0.1)

  session.execute(batch)

except Exception as e:
    print(e)
    print('An error occured')

else:
  print('\n')
  print(str(len(fruits_list)) + ' rows added to the cart_products table with Success!')

print('========================================')

# **READ** statement

In [ ]:
print('========================================')

try:

    output = session.execute(
        "SELECT * FROM cart_products"
    )
    for row in output:
       print('Product Name:', row.product_name, 'Quantity:', row.quantity,'Time:', row.product_timestamp, 'Description:', row.product_description)

except Exception as e:
    print(e)
    print('An error occured')


print('========================================')

In [ ]:
print('========================================')

try:

    output = session.execute(
        "SELECT * FROM cart_products WHERE cart_id  = %s and product_name = %s",
        [cart_id, 'Banana']
    )
    for row in output:
       print('Product Name:', row.product_name, 'Quantity:', row.quantity,'Time:', row.product_timestamp, 'Description:', row.product_description)

except Exception as e:
    print(e)
    print('An error occured')


print('========================================')

# **UPDATE** statement

In [ ]:
product_description = 'Curved, potassium-packed chuckle fruit'

print("========================================")

try:

    session.execute(
        "UPDATE cart_products SET product_description=%s WHERE cart_id= %s AND product_name= %s",
        [product_description, cart_id, 'Banana' ])

except Exception as e:
    print(e)
    print('An error occured')
else:
    print('Product description updated with Success!')


print('========================================')

In [ ]:
print('========================================')

try:

    output = session.execute(
        "SELECT * FROM cart_products WHERE cart_id  = %s and product_name = %s",
        [cart_id, 'Banana']
    )
    for row in output:
       print('Product Name:', row.product_name, 'Quantity:', row.quantity,'Time:', row.product_timestamp, 'Description:', row.product_description)

except Exception as e:
    print(e)
    print('An error occured')


print('========================================')

# **DELETE** statement

In [ ]:
print("========================================")

try:

    session.execute(
        "DELETE FROM cart_products WHERE cart_id= %s AND product_name= %s",
        [cart_id, 'Apple' ])

except Exception as e:
    print(e)
    print('An error occured')
else:
    print('Row deleted with Success!')

print("========================================")

# **EDA** with pandas

In [ ]:
import pandas as pd

In [ ]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

query = "select * from cart_products"
rslt = session.execute(query, timeout=None)
df = rslt._current_rows

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt

data = df.sort_values(by='product_timestamp')
plt.figure(figsize=(10, 6))
plt.plot(data['product_timestamp'], data['quantity'])
plt.title('Time vs. Quantity')
plt.xlabel('Product Timestamp')
plt.ylabel('Quantity')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
